In [1151]:
import pandas as pd
import numpy as np

In [1152]:
data = pd.read_excel('./data/example_data_cleaning.xlsx')

df = pd.DataFrame(data)
df.head(5)

,TransactionID,ClientID,BirthYear,Amount,Profession,Department,Risk
0,1,34982,2013.0,12900,Student,78,Low
1,2,34983,2015.0,1230,barmen,78,High
2,3,34984,1945.0,3450,student,78,Medium
3,4,34985,1923.0,5670,manager,78,Low
4,5,34986,1978.0,7890,hr,78,High


In [1153]:
#1.	Provide the descriptive statistics for the dataset
stat = df.describe()
stat

,TransactionID,ClientID,BirthYear,Amount,Department
count,30.000000,30.000000,28.000000,3.000000e+01,30.0
mean,15.500000,34993.833333,1968.316327,1.010097e+05,78.0
std,8.803408,8.132876,27.963867,4.342616e+05,0.0
min,1.000000,34982.000000,1923.000000,1.230000e+03,78.0
25%,8.250000,34987.250000,1944.714286,1.233000e+04,78.0
50%,15.500000,34991.500000,1967.000000,1.899000e+04,78.0
75%,22.750000,34999.750000,1988.000000,2.953500e+04,78.0
max,30.000000,35008.000000,2017.000000,2.399090e+06,78.0


In [1154]:
# Department has a single value 78, so we can drop it.
df.drop("Department", axis = 1, inplace = True)

In [1155]:
#2.	Calculate the number of missing values.
mv=df.isna().sum()
mv

TransactionID    0
ClientID         0
BirthYear        2
Amount           0
Profession       2
Risk             0
dtype: int64

In [1156]:
# TransactionID

# Check if all values are unique
df["TransactionID"].is_unique

# True: OK with TransactionID

# Make it index
df.set_index("TransactionID", inplace = True)

In [1157]:
#1. BirthYear

df1 = df.copy()

In [1158]:
#1.1. Completeness

# Extract list of ClientID with missing birth years
clientid_missing_birthyear = list(df1.loc[df1["BirthYear"].isna(), "ClientID"])

# Complete missing birth years with max BirthYear for the same clientid
for clientid in clientid_missing_birthyear:
    # Get max BirthYear
    max_birthyear = df1.loc[df1["ClientID"] == clientid]["BirthYear"].max()
    # Fill missing values
    df1.loc[(df1["ClientID"] == clientid) & (df1["BirthYear"].isna()), "BirthYear"] = max_birthyear
    
# Check if BirthYear is now complete
print(df1["BirthYear"].isna().sum() == 0)

True


In [1159]:
#1.2. Consistency  

# Extract list of ClientID with more than a single birth year
nb_birthyears_clientid = df1.groupby("ClientID")['BirthYear'].nunique() > 1
clientid_several_birthyears = set(nb_birthyears_clientid[nb_birthyears_clientid == True].index)

# Complete with max mode (if several modes)
for clientid in clientid_several_birthyears:
    # Get mode
    maxmode_birthyear = df1.loc[df1["ClientID"] == clientid]["BirthYear"].mode().max()
    # Fill values
    df1.loc[(df1["ClientID"] == clientid) & (df1["BirthYear"] != maxmode_birthyear), "BirthYear"] = maxmode_birthyear

#Check if BirthYear is now consistent
print(all(df1.groupby("ClientID")['BirthYear'].nunique() == 1))

True


In [1160]:
#1.3. Outliers

stats_df1 = df1.describe().transpose()
stats_df1["IQR"] = stats_df1['75%'] - stats_df1['25%']
cutoff_birthyear = stats_df1.loc["BirthYear", "IQR"] * 0.5
lower_year = stats_df1.loc["BirthYear", "25%"] - cutoff_birthyear
upper_year = stats_df1.loc["BirthYear", "75%"] + cutoff_birthyear

# ~ to invert mask
outliers_birthyear = df1.loc[~(df1["BirthYear"].between(lower_year, upper_year))]
df1.drop(outliers_birthyear.index, axis = 0, inplace = True)

In [1161]:
#2. Profession

df2 = df1.copy()

In [1162]:
#2.1. Completeness

# Extract list of ClientID with missing Profession
clientid_missing_prof = set(df2.loc[df2["Profession"].isna(), "ClientID"])

# Complete missing birth years with mode Profession for the same clientid
for clientid in clientid_missing_prof:
    # Get mode Profession
    mode_prof = list(df2.loc[df2["ClientID"] == clientid]["Profession"].mode(dropna = True))
    # Fill missing values
    df2.loc[(df2["ClientID"] == clientid) & (df2["Profession"].isna()), "Profession"] = mode_prof[0]

# Check if Profession is now complete
print(df1["BirthYear"].isna().sum() == 0)

True


In [1163]:
#2.2. Consistency

# Extract list of ClientID with more than a single profession
nb_prof_clientid = df2.groupby("ClientID")['Profession'].nunique() > 1
clientid_several_profs = set(nb_prof_clientid[nb_prof_clientid == True].index)

# Complete with mode
for clientid in clientid_several_profs:
    # Get mode Profession
    mode_prof = df2.loc[df2["ClientID"] == clientid]["Profession"].mode()
    # Fill values with first mode
    df2.loc[(df2["ClientID"] == clientid) & (df2["Profession"] != mode_prof[0]), "Profession"] = mode_prof[0]


#Check if Profession is now consistent
print(all(df2.groupby("ClientID")['Profession'].nunique() == 1))

True


In [1164]:
#2.3. Homogeneity

# Dictionary of Profession: goal is to get it in the form {original_prof:clean_prof}

# Original dictionary
prof_dict = {prof:prof for prof in df2.Profession.unique()}
prof_dict

{'student': 'student',
 'hr': 'hr',
 'Manager': 'Manager',
 'HR': 'HR',
 'BDM': 'BDM',
 'researcher': 'researcher',
 'professor': 'professor',
 'Hairdresser': 'Hairdresser',
 'etudient': 'etudient',
 'barmen': 'barmen',
 'manager': 'manager',
 'bdm': 'bdm'}

In [1165]:
# The same professions have various case formats, let's make them all lower cases
prof_dict = {org_prof:new_prof.lower() for (org_prof,new_prof) in prof_dict.items()}
prof_dict

{'student': 'student',
 'hr': 'hr',
 'Manager': 'manager',
 'HR': 'hr',
 'BDM': 'bdm',
 'researcher': 'researcher',
 'professor': 'professor',
 'Hairdresser': 'hairdresser',
 'etudient': 'etudient',
 'barmen': 'barmen',
 'manager': 'manager',
 'bdm': 'bdm'}

In [1166]:
# Profession 'student' is also referenced by 'etudiant' and 'etudient'
# Let's make it all 'student'
prof_dict['etudiant'] = 'student'
prof_dict['etudient'] = 'student'
set(prof_dict.values())

{'barmen',
 'bdm',
 'hairdresser',
 'hr',
 'manager',
 'professor',
 'researcher',
 'student'}

In [1167]:
# Feed new professions back to dataframe
df2["Profession"] = df2["Profession"].map(prof_dict)

In [1168]:
#3. Amount

df3 = df2.copy()

In [1169]:
#3.1 Outliers
stats_df3 = df3.describe().transpose()
stats_df3["IQR"] = stats['75%'] - stats['25%']
cutoff_amt = stats_df3.loc["Amount", "IQR"] * 3
lower_amt = stats_df3.loc["Amount", "25%"] - cutoff_amt
upper_amt = stats_df3.loc["Amount", "75%"] + cutoff_amt

# ~ to invert mask
outliers_amt = df3.loc[~(df3["Amount"].between(lower_amt, upper_amt) )]
outliers_amt
# No outlier to drop

,ClientID,BirthYear,Amount,Profession,Risk
TransactionID,,,,,


In [1170]:
#4. Risk

df4 = df3.copy()

In [1171]:
#4.1. Homogeneity
set(df4.Risk)
# {'High', 'Low', 'Medium'} is homogeneous

{'High', 'Low', 'Medium'}

In [1172]:
#4.2. Consistency
# Risk is by transaction, not by client, so no consistency check with available data

In [1173]:
#5. Encoding categorical columns

df5 = df4.copy()

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

# Label Encoder for "Profession"
encoder1 = LabelEncoder()
df5["Profession_encode"]=encoder1.fit_transform(df5["Profession"])
df5.drop(columns=["Profession"], inplace = True)

# Ordinal Encoder for "Risk"

encoder2 = OrdinalEncoder(categories=[['Low', 'Medium', 'High']])
df5["Risk_encode"]=encoder2.fit_transform(df5.loc[:,["Risk"]])
df5.drop(columns=["Risk"], inplace = True)

    

df5.head(5)

,ClientID,BirthYear,Amount,Profession_encode,Risk_encode
TransactionID,,,,,
3,34984,1945.0,3450,7,1.0
5,34986,1978.0,7890,3,2.0
6,34987,1967.0,10110,4,1.0
15,34987,1967.0,30090,4,0.0
22,34987,1967.0,18990,4,2.0


In [1174]:
#6. Save final database as CSV
df_clean = df5.copy()
df_clean.to_csv("./data/example_data_cleaning_CLEANED.csv")